In [23]:
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras import Input
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

In [24]:
#Constants
NUM_FOLDS = 5

In [25]:
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 

# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets
y = y.rename(columns={"num":"diagnosis"})

#Due to the low number of samples, a k-fold validation scheme will be used
fold_split = int(len(X)/NUM_FOLDS)

In [26]:
#Normalizes data
X = (X-X.mean())/X.std()

In [32]:
#Creates model
def neural_net(input_shape=13, output_shape=5):
    model = Sequential( [
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(output_shape, activation='softmax')
        ])
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['F1Score'])
    return model


In [33]:
f1scores = []

for i in range(0,NUM_FOLDS):
    print(f"Starting fold {i}...")
    fold_start = i*fold_split
    fold_end = fold_start + fold_split

    X_val = X[fold_start:fold_end]
    y_val = y[fold_start:fold_end]

    X_train = pd.concat([X[:fold_start], X[fold_end:]], axis=0, ignore_index=True)
    y_train = pd.concat([y[:fold_start], y[fold_end:]], axis=0, ignore_index = True)

    nn = neural_net()
    nn.fit(X_train, y_train, epochs = 100, batch_size = 20, verbose=0)

    y_pred = np.argmax(nn.predict(X_val), axis=1)
    f1 = f1_score(y_val, y_pred, average='weighted')
    f1scores.append(f1)
    print(f"Fold {i} f1-score: {f1}")

Starting fold 0...


NotImplementedError: Iterating over a symbolic KerasTensor is not supported.

In [22]:
mean_f1 = np.average(f1scores)
print(f"Average f1-score: {mean_f1}")

Average f1-score: 0.38308484108216795
